### Groq Client and relevant imports


In [1]:
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
load_dotenv()
client = Groq()
# Groq uses the same API signature as OpenAI.
# FYI: LiteLLM is a LLM Gateway to provide model access, logging and usage tracking across 100+ LLMs. All in the OpenAI format. https://www.litellm.ai/

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [3]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt.",
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [4]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [5]:
user_code = (
    client.chat.completions.create(
        messages=generation_chat_history, model="llama3-70b-8192"
    )
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [6]:
display_markdown(user_code, raw=True)

Here is a basic implementation of requesting an API using the Requests library in Python:
```
import requests

def request_api(url, method='GET', params=None, data=None, headers=None):
    """
    Send a request to the specified API endpoint

    Args:
        url (str): The URL of the API endpoint
        method (str, optional): The HTTP method to use (default: 'GET')
        params (dict, optional): URL parameters to include in the request
        data (dict, optional): Data to include in the request body
        headers (dict, optional): Custom headers to include in the request

    Returns:
        response (requests.Response): The response from the API
    """
    try:
        response = requests.request(method, url, params=params, data=data, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

    return response

# Example usage:
url = "https://api.example.com/endpoint"
params = {"key": "value"}
headers = {"Authorization": "Bearer YOUR_API_KEY"}

response = request_api(url, params=params, headers=headers)

if response:
    print(response.json())  # Print the response JSON
```
This implementation provides a basic wrapper around the Requests library's `request` method, allowing you to specify the URL, method, parameters, data, and headers for the request. It also includes some basic error handling to catch any exceptions raised by the Requests library.

Please let me know if you have any specific requirements or constraints for the implementation, or if you'd like me to revise anything!

## Reflection Step


This is equivalent to asking a followup question in say ChatGPT


In [7]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [8]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.

In [9]:
critique = (
    client.chat.completions.create(
        messages=reflection_chat_history, model="llama3-70b-8192"
    )
    .choices[0]
    .message.content
)

In [10]:
# Display

display_markdown(critique, raw=True)

The implementation you provided is a good start, but there are a few areas where it can be improved. Here are some suggestions:

1. **Error Handling:** The current error handling is quite basic. Instead of just printing the error message, consider logging the error using a logger like `logger.error(str(e))`. This way, you can configure the logging to log errors to a file or a centralized logging system.

2. **Returning None:** When an error occurs, the function returns `None`. This can lead to issues if the caller doesn't check for `None` before trying to access the response. Consider raising an exception instead, or returning a custom error object.

3. **JSON Parsing:** The `response.json()` call can raise a `ValueError` if the response is not JSON. You should handle this exception. 

4. **Timeout:** The `requests` library has a default timeout of none, which means that it will wait indefinitely for a response. This can lead to your program hanging if the API is not responding. Consider adding a timeout parameter to your function and passing it to the `requests.request` call.

5. **Type Hints:** Your function is missing type hints for the return value and the `response` variable. 

Here's an updated version of your code:

```
import requests
import logging

def request_api(url: str, method: str = 'GET', params: dict = None, data: dict = None, headers: dict = None, timeout: int = 10) -> requests.Response:
    """
    Send a request to the specified API endpoint

    Args:
        url (str): The URL of the API endpoint
        method (str, optional): The HTTP method to use (default: 'GET')
        params (dict, optional): URL parameters to include in the request
        data (dict, optional): Data to include in the request body
        headers (dict, optional): Custom headers to include in the request
        timeout (int, optional): The timeout in seconds for the request (default: 10)

    Returns:
        response (requests.Response): The response from the API
    """
    logger = logging.getLogger(__name__)

    try:
        response = requests.request(method, url, params=params, data=data, headers=headers, timeout=timeout)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        logger.error(f"Error: {e}")
        raise  # Reraise the exception

    try:
        return response
    except AttributeError:
        logger.error("Failed to get response")
        raise  # Reraise the exception

# Example usage:
url = "https://api.example.com/endpoint"
params = {"key": "value"}
headers = {"Authorization": "Bearer YOUR_API_KEY"}

try:
    response = request_api(url, params=params, headers=headers)
    if response:
        try:
            print(response.json())  # Print the response JSON
        except ValueError:
            logger.error("Failed to parse JSON response")
except Exception as e:
    logger.error(f"Failed to request API: {e}")
```

This updated code includes:

*   Better error handling using a logger and re-raising exceptions
*   A timeout parameter to prevent hanging
*   Type hints for the return value and the `response` variable
*   Handling of a `ValueError` when parsing the JSON response

Add CRITIQUE to chat...


In [11]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [12]:
essay = (
    client.chat.completions.create(
        messages=generation_chat_history, model="llama3-70b-8192"
    )
    .choices[0]
    .message.content
)

In [13]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for the suggestions! Your updated code indeed improves the implementation in several ways. Here's a revised version that incorporates your suggestions and provides additional improvements:

```
import requests
import logging
from typing import Union

def request_api(url: str, method: str = 'GET', params: dict = None, data: dict = None, headers: dict = None, timeout: int = 10) -> Union[requests.Response, None]:
    """
    Send a request to the specified API endpoint

    Args:
        url (str): The URL of the API endpoint
        method (str, optional): The HTTP method to use (default: 'GET')
        params (dict, optional): URL parameters to include in the request
        data (dict, optional): Data to include in the request body
        headers (dict, optional): Custom headers to include in the request
        timeout (int, optional): The timeout in seconds for the request (default: 10)

    Returns:
        response (requests.Response or None): The response from the API, or None if an error occurs
    """
    logger = logging.getLogger(__name__)

    try:
        response = requests.request(method, url, params=params, data=data, headers=headers, timeout=timeout)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        logger.error(f"Error: {e}")
        return None

    try:
        return response
    except AttributeError:
        logger.error("Failed to get response")
        return None

# Example usage:
url = "https://api.example.com/endpoint"
params = {"key": "value"}
headers = {"Authorization": "Bearer YOUR_API_KEY"}

response = request_api(url, params=params, headers=headers)

if response:
    try:
        print(response.json())  # Print the response JSON
    except ValueError:
        logging.error("Failed to parse JSON response")
else:
    logging.error("Failed to request API")
```

The key changes I made include:

1.  I added a `Union` type hint to indicate that the function can return either a `requests.Response` object or `None`.
2.  I removed the `raise` statement in the `except` block and instead returned `None` to indicate that an error occurred. This allows the caller to handle the error accordingly.
3.  I added a check to see if `response` is not `None` before trying to access its `json()` method.
4.  I used the `logging` module to log errors instead of printing them. This allows for more flexibility in configuring the logging behavior.

Overall, this revised implementation provides better error handling, improved type hints, and more robust logging.

<br>

## We can of course make this a Class...
